In [694]:
import pandas as pd
import missingno as msno
import re

In [695]:
dff = pd.read_csv("sakammm_knigi.csv", encoding='utf-8-sig')
df = dff.copy()

In [696]:
# df.head()

In [697]:
# df.isnull().sum()

### Clean Author Column

In [698]:
# df['Author'] = df['Author'].str.replace("од:", "")

In [699]:
# df.isnull().sum()

### Duplicates
- 10 entries are completely identical. We can merge the, Keep only one

In [700]:
# duplicates = df[df.duplicated(keep=False)]
# duplicates = duplicates.sort_values(by=['Title', 'Author'])
# print("Length of duplicates: ", len(duplicates))
# duplicates.head(10)

In [701]:
# print("Original:", len(df))
# df = df.drop_duplicates()
# print("After drop_duplicates:", len(df))
# print("Duplicates removed:", len(df) - len(df))

### Author

- All 7 Harry Poter books have a missing value in the Author field. Replace all with  'Џ.К. Роулинг'

In [702]:
# harry_potter_rows = df[df['Title'].str.contains('Хари Потер', case=False, na=False)]
# harry_potter_rows

In [703]:
# df.loc[df['Title'].str.contains('Хари Потер', case=False, na=False), 'Author'] = 'Џ.К. Роулинг'

- Extract authors from the titles that contain authors(e.g. "Избрани песни – Ана Ахматова")


In [704]:
# # See which authord have – or  - or —
# authors_fix = df[df['Title'].str.contains(r'\s[–\-—]\s\w+(?:\s\w+)?', regex=True, case=False)]
# authors_fix.head(100)

In [705]:
# updates = {
#    521: {'Title': 'Избрани песни', 'Author': 'Ана Ахматова'},
#    770: {'Title': 'Подарокот', 'Author': 'Едит Егер'},
#    843: {'Title': 'Херој', 'Author': 'Р.Бирн'},
#    976: {'Title': 'ИСТОРИЈА НА РЕЛИГИИТЕ', 'Author': 'Александар Мењ'}
# }

# for idx, update in updates.items():
#     df.at[idx, 'Title'] = update['Title']
#     df.at[idx, 'Author'] = update['Author']

In [706]:
# df[df['Title'].str.contains('ИСТОРИЈА НА РЕЛИГИИТЕ', case=False)]
# df[df['Title'].str.contains('Херој', case=False)]
# df[df['Title'].str.contains('Избрани песни', case=False)]
# df[df['Title'].str.contains('Подарокот', case=False)]

Check for 'и'

In [707]:
# two_authors_df = df[df['Author'].str.contains(r'\sи\s', case=False, na=False)]
# two_authors_df

In [708]:
# two_authors_df = df[df['Author'].str.contains(r'\sи\s', case=False, na=False)].copy()

# two_authors_df['Author'] = two_authors_df['Author'].str.split(' и ')

# exploded_df = two_authors_df.explode('Author', ignore_index=True)

# df = pd.concat([df[~df['Author'].str.contains(r'\sи\s', case=False, na=False)], exploded_df], ignore_index=True)

- Check for Authors divided by , Example: 1060	Аурис	 Себастијан Фицек, Винсент Клиш,

In [709]:
# # Find rows with commas in Author
# comma_rows = df['Author'].str.contains(r'\s*,\s*', na=False)

# # Remove trailing commas in those rows
# df.loc[comma_rows, 'Author'] = df.loc[comma_rows, 'Author'].str.rstrip(', ')


In [710]:
# comma_authors_df = df[df['Author'].str.contains(r'\s*,\s*', na=False)]
# comma_authors_df

In [711]:
# comma_authors_df = df[df['Author'].str.contains(r'\s*,\s*', na=False)].copy()

# comma_authors_df['Author'] = comma_authors_df['Author'].str.rstrip(', ')
# comma_authors_df['Author'] = comma_authors_df['Author'].str.split(r'\s*,\s*')
# exploded_comma_df = comma_authors_df.explode('Author', ignore_index=True)
# df = df[~df['Author'].str.contains(r'\s*,\s*', na=False)]
# df = pd.concat([df, exploded_comma_df], ignore_index=True)


In [712]:
# comma_authors_df = df[df['Author'].str.contains(r'\s*,\s*', na=False)]
# comma_authors_df

In [713]:
# df.to_csv("sakammm_knigi.csv", encoding='utf-8-sig', index=False)
# df = pd.read_csv("sakammm_knigi.csv", encoding='utf-8-sig')

- Find titles that appear under multiple authors <br/> The books from before have multiple authors: (Кралицата Шарлот, Да можев да ти го подарам Божиќ, Аурис, Совршената замена)

In [715]:
# Group by Title and collect unique authors
title_authors = df.groupby('Title')['Author'].unique()

# Reset to a DataFrame
title_authors_df = title_authors.reset_index()

# Add a count of how many authors per title
title_authors_df['Author Count'] = title_authors_df['Author'].apply(len)

# Filter titles with more than one author (i.e., different authors for the same book title)
duplicate_titles = title_authors_df[title_authors_df['Author Count'] > 1]

# Sort by number of authors if desired
duplicate_titles = duplicate_titles.sort_values(by='Author Count', ascending=False)

# Show the result
duplicate_titles.head(200)


,Title,Author,Author Count
1140,Подарокот,"[Едит Егер, Себастијан Фицек, nan]",3
264,Вистинскиот,"[ Џон Марс, nan]",2
126,Аурис,"[ Себастијан Фицек, Винсент Клиш]",2
361,Да можев да ти го подарам Божиќ,"[ Џенифер Л. Мајер, Лин Плорд]",2
761,Кралицата Шарлота,"[ Џулија Квин, Шонда Рајмс]",2
1420,Совршената замена,"[ Грир Хендрикс, Сара Пеканен]",2
1604,Херој,"[ Саманта Јанг, Р.Бирн]",2
